## Day 20: Particle Swarm
    
http://adventofcode.com/2017/day/20

### Part 1

The answer here is the particle with the smallest acceleration magnitude. If two particles with different accelerations start from completely arbitrary finite locations and velocities, then at some point the particle with the higher acceleration will have higher velocity and continue to have higher velocity, so then at some possibly later point will be diverging further and faster from the origin. 

The input format is a bit awkward, so I've used Emacs to turn it into csv.

In [1]:
import numpy as np

def split_by_length(xs, length):
    l_xs = list(xs)
    return [l_xs[i:i+length] for i in range(0, len(l_xs), length)]

        
with open('input.csv') as f:
    particles = [np.array(v) for v in [split_by_length(map(int, line.strip().split(',')), 3) 
                                       for line in f]]
    
particles[0]

array([[ -717, -4557,  2578],
       [  153,    21,    30],
       [   -8,     8,    -7]])

Using Manhattan vectors is a bit odd, but I can't see why it wouldn't work for this. The particle having acceleration with the largest Manhattan magnitude will still move further away than others when using Manhattan distance as a measure of distance.

In [2]:
def manhattan_magnitude(v):
    return sum(abs(v_i) for v_i in v)

def acceleration_magnitude(p):
    return manhattan_magnitude(p[2])

acceleration_magnitude(particles[0])

23

In [3]:
min(range(len(particles)), key=lambda i: acceleration_magnitude(particles[i]))

115

In [4]:
particles[115]

array([[-129, 1190, -665],
       [   3,  -82,   41],
       [   1,    0,    1]])

That looks small, let's give it a go.

It's the wrong answer. What am I missing? 

In [5]:
for p in sorted(particles, key=acceleration_magnitude)[:10]:
    print(p)

[[-129 1190 -665]
 [   3  -82   41]
 [   1    0    1]]
[[  528 -1788 -1536]
 [    3    77    65]
 [   -2     0     0]]
[[  692  -175 -1125]
 [  -38    19    49]
 [    0    -1     1]]
[[  409   864 -1345]
 [    1   -28    32]
 [   -1     0     1]]
[[-1085  -561  1531]
 [   41    -8   -56]
 [    0     2     0]]
[[1119 -358 -696]
 [ -83   20   46]
 [   2    0   -1]]
[[-940 1847 1177]
 [  45  -71  -30]
 [   0   -1   -2]]
[[ 1720   726 -1414]
 [  -33   -28    64]
 [   -2     0    -1]]
[[ -932   143 -1423]
 [   50   -10   107]
 [    1     0    -2]]
[[-2370  3605   -22]
 [   91  -112    15]
 [    0    -2    -1]]


Of course, I need to distinguish between particles with the same acceleration. For two particles with the same acceleration one with a velocity that minimises the disruption to the acceleration will travel furthest. This can be measured by the magnitude of the acceleration added to the original velocity.

<i>Slightly</i> more formally, if $p_t$ is the position at time zero, $v_t$ the velocity, and $a$ the constant acceleration, then
$$
\begin{array}{lllll}
p_0 & = & p_0 & \\
p_1 & = & p_0 + (v_0 + a) & = & p_0 + v_0 + a \\
p_2 & = & p_0 + (v_0 + a) + (v_0 + 2a) & = & p_0 + 2v_0 + 3a \\
p_3 & = & p_0 + (v_0 + a) + (v_0 + 2a) & = & p_0 + 3v_0 + 6a \\
& \vdots \\
p_t & = & p_0 + t.v_0 + \frac{t(t+1)}{2}a
\end{array}
$$
The coefficient of $a$ is $\frac{t+1}{2}$ larger than that of $v_0$, so as $t$ becomes sufficiently large the acceleration value dominates, however small it is, and the initial velocity determines which will have travelled further amongst particles with equal acceleration. 

In [6]:
def velocity_magnitude(p):
    return manhattan_magnitude(p[1])

min(range(len(particles)), 
    key= lambda i: (acceleration_magnitude(particles[i]), 
                    manhattan_magnitude(particles[i][2] + particles[i][1])))

344

In [7]:
particles[344]

array([[  409,   864, -1345],
       [    1,   -28,    32],
       [   -1,     0,     1]])

That's the right answer.

### Part 2

For each pair of particles $p_i$ and $p_j$ find a positive integer $t$ such that
$$ p_{i0} + t.v_{i0} + \frac{t(t+1)}{2}a_i = p_{j1} + t.v_{j1} + \frac{t(t+1)}{2}a_j $$

Rewrite as quadilateral
$$ \frac{a_i - a_j}{2}t^2 + (\frac{a_i - a_j}{2} + v_{i0} - v_{j0})t + p_{i0} - p_{j0} = 0$$
and solve for each dimension; if there's a common root then the particles collide. Then record the time and place the particles collided in case one of them is destroyed earlier.

In [8]:
import math
import itertools
from collections import defaultdict


def position(particle, t):
    p, v, a = particle
    return p + v * t + a * t * (t + 1) // 2


def near_ints(floats, tolerance=0.0001):
    # For a sequence of floats return ints if they're
    # within a margin of tolerance in case of floating
    # point errors in the maths
    for f in floats:
        if abs(f - math.ceil(f)) <= tolerance:
            yield math.ceil(f)
        elif abs(f - math.floor(f)) <= tolerance:
            yield math.floor(f)
            
            
def solve_quad(a, b, c):
    if a == 0:
        if b != 0:
            return {-c/b}
        else:
            return {} # deal with all values == 0 below
    elif b**2 >= 4*a*c: 
        return {(-b + math.sqrt(b**2 - 4*a*c))/(2*a), (-b - math.sqrt(b**2 - 4*a*c))/(2*a)}
    else: # not interested in imaginary results
        return {}
    
    
def solve_eqn(p, v, a):
    # t must be greater than zero
    return {t for t in near_ints(solve_quad(a/2, a/2 + v, p)) if t > 0}


def collision(particle_i, particle_j):
    particle_diff = particle_i - particle_j
    p, v, a = particle_diff
    
    # Need to find a root (or roots) common to all dimensions d
    # (possible to have two roots if travelling on same path).
    # If all p, v, a values for a dimension are 0 then ignore
    # as it solves trivially to any value of t.
    for root in set.intersection(*[solve_eqn(p[d], v[d], a[d]) for d in range(3)
                                   if p[d] != 0 or v[d] != 0 or a[d] != 0]):
        yield root
    

def collisions(particles):
    # Returns a dictionary with keys (time, position) tuples and 
    # values a set of particles colliding at that time
    all_collisions = defaultdict(set)
    
    for i, j in itertools.combinations(range(len(particles)), 2):
        for t in collision(particles[i], particles[j]):
            all_collisions[(t, tuple(position(particles[i], t)))].update({i, j})
            
    return all_collisions

In [9]:
test_data = '''-6,0,0, 3,0,0, 0,0,0    
-4,0,0, 2,0,0, 0,0,0
-2,0,0, 1,0,0, 0,0,0
3,0,0, -1,0,0, 0,0,0'''.splitlines()

test_particles = [np.array(v) for v in [split_by_length(map(int, line.strip().split(',')), 3) 
                                        for line in test_data]]

test_collisions = collisions(test_particles)
test_collisions

defaultdict(set, {(2, (0, 0, 0)): {0, 1, 2}})

That's correct for the test data, no doubt there'll be more edge cases in the given data.

In [10]:
%time possible_collisions = collisions(particles)

CPU times: user 15.6 s, sys: 268 µs, total: 15.6 s
Wall time: 15.6 s


In [11]:
possible_collisions

defaultdict(set,
            {(10, (-23, -16, 7)): {467, 468, 469, 470, 471},
             (10, (-12, -44, 7)): {255,
              256,
              257,
              258,
              259,
              260,
              261,
              262,
              263},
             (10, (-5, -1, -50)): {417, 418, 419, 420, 421, 422},
             (10, (44, 1, 3)): {129, 130, 131, 132, 133, 134},
             (11, (-31, 2, -7)): {423, 424, 425, 426},
             (11, (-13, -10, -33)): {400, 401, 402, 403, 404, 405, 406},
             (11, (-7, 49, -2)): {583, 584, 585, 586, 587, 588, 589, 590, 591},
             (11, (-5, -9, -24)): {89, 90, 91},
             (11, (23, -38, 43)): {380, 381, 382, 383},
             (12, (-5, -28, 26)): {51, 52, 53, 54, 55},
             (14, (-5, -24, 27)): {56, 57, 58, 59, 60},
             (14, (18, 42, 14)): {112, 113, 114, 115, 116},
             (14, (46, 36, 20)): {535, 536, 537, 538, 539},
             (15, (-85, 19, 26)): {503, 504, 505, 506, 5

We can now get a dictionary of all possible collisions, where we have a list of particles colliding at each time and place. The function below runs through the potential collisions in order of which will be earliest, tracking which particles are still going. At each potential collision if there are multiple particles that haven't already collided and being destroyed, they will be removed from the list of those still going. After all possible collisions the set of those particles still going is returned.

In [12]:
def process_collisions(particles):
    possible_collisions = collisions(particles)
    uncollided = set(range(len(particles)))
    
    # Sort the collisions by time so the first
    # aren't taken into account
    for k in sorted(possible_collisions):
        colliding_here = possible_collisions[k].intersection(uncollided)
        
        # If the number of colliders is 1 at this time and place then
        # leave it unharmed as there is nothing to collide with
        if len(colliding_here) > 1:
            uncollided -= colliding_here
            
    return uncollided

Let's try for the test data.

In [13]:
process_collisions(test_particles)

{3}

That's correct. Now for the given data.

In [14]:
%time uncollided = process_collisions(particles)

CPU times: user 15.6 s, sys: 8 µs, total: 15.6 s
Wall time: 15.6 s


In [15]:
uncollided

{596,
 597,
 598,
 599,
 600,
 601,
 602,
 603,
 604,
 605,
 606,
 607,
 608,
 609,
 610,
 611,
 612,
 613,
 614,
 615,
 616,
 617,
 618,
 619,
 620,
 621,
 622,
 623,
 624,
 625,
 626,
 627,
 628,
 629,
 630,
 631,
 632,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 661,
 662,
 663,
 664,
 665,
 666,
 667,
 668,
 669,
 670,
 671,
 672,
 673,
 674,
 675,
 676,
 677,
 678,
 679,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 701,
 702,
 703,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 730,
 731,
 732,
 733,
 734,
 735,
 736,
 737,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 755,
 756,
 757,
 758,
 759,
 760,
 761,
 762

In [16]:
len(uncollided)

404

That's correct.

### Post-mortem

All possible collisions occurred within 40 ticks so simulating it until nothing was happening would have worked in reasonable time and I wouldn't have needed to dust off t. Oh well.